In [1]:
"""
Created on Mon Apr 08 16:22 2024

This script is to make a timeseries of runs that have crossed the viability limit + compare to hydrofracturing

@author: Clara Burgard
"""

'\nCreated on Mon Apr 08 16:22 2024\n\nThis script is to make a timeseries of runs that have crossed the viability limit + compare to hydrofracturing\n\n@author: Clara Burgard\n'

In [1]:
import xarray as xr
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import cm
import matplotlib as mpl
import cmocean
import glob
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
import os

In [2]:
#sns.set_context('poster')
sns.set_context('paper')

In [3]:
%matplotlib qt5

QStandardPaths: XDG_RUNTIME_DIR not set, defaulting to '/tmp/runtime-burgardc'


In [4]:
# make the domain a little smaller to make the computation even more efficient - file isf has already been made smaller at its creation
map_lim = [-3000000,3000000]

READ IN DATA

In [5]:
home_path = '/bettik/burgardc/'
plot_path = '/bettik/burgardc/PLOTS/summer_paper_plots/'
outputpath_GL = '/bettik/burgardc/DATA/SUMMER_PAPER/processed/GL_FLUX/'
inputpath_weights = '/bettik/burgardc/DATA/SUMMER_PAPER/processed/ANALYSIS/'
inputpath_atmo = '/bettik/burgardc/DATA/SUMMER_PAPER/raw/TS_SMB_DATA/out/'
inputpath_data='/bettik/burgardc/DATA/SUMMER_PAPER/interim/'


In [6]:
inputpath_mask = home_path+'/DATA/SUMMER_PAPER/interim/ANTARCTICA_IS_MASKS/BedMachine_4km/'
file_isf_orig = xr.open_dataset(inputpath_mask+'BedMachinev2_4km_isf_masks_and_info_and_distance_oneFRIS.nc')
nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
rignot_isf = file_isf_nonnan.Nisf.where(np.isfinite(file_isf_nonnan['isf_area_rignot']), drop=True)
file_isf = file_isf_nonnan.sel(Nisf=rignot_isf)

In [7]:
weight_file = xr.open_dataset(inputpath_weights + 'bayesian_weights_davison_varying_combined_withoutGISS.nc')
weight_2300_file = xr.open_dataset(inputpath_weights + 'bayesian_weights_davison_varying_combined_2300_withoutGISS.nc')
file_viability_info = xr.open_dataset(inputpath_weights + 'all_fluxes_br_withoutGISS.nc')

In [8]:
### Colorbar:
cmap = mpl.cm.YlOrRd # define the colormap
# extract all colors from the .jet map
cmaplist = [cmap(i) for i in range(cmap.N)]
# force the first color entry to be grey
# cmaplist[0] = (.5, .5, .5, 1.0)

# create the new map
cmap_new = mpl.colors.LinearSegmentedColormap.from_list(
    'Custom cmap', cmaplist, cmap.N)

alpha = 0.8
cmap_with_alpha = cmap(np.arange(cmap_new.N))
cmap_with_alpha[:, -1] = alpha
cmap_new2 = mpl.colors.ListedColormap(cmap_with_alpha)

# define the bins and normalize
bounds = np.arange(6)
norm = mpl.colors.BoundaryNorm(bounds, cmap_new2.N)

In [33]:
rgba = cmap(0.33)
print(rgba)

(0.996078431372549, 0.7538177623990773, 0.35810841983852365, 1.0)


Air temperature

In [9]:
inputpath_tas = '/bettik/burgardc/DATA/SUMMER_PAPER/interim/CMIP_TEMP/'
mod_list = ['ACCESS-CM2','ACCESS-ESM1-5','CNRM-CM6-1','CNRM-ESM2-1','GISS_E2-1-H',
            'CESM2-WACCM','CESM2','CanESM5','IPSL-CM6A-LR','MRI-ESM2-0','MPI-ESM1-2-HR',
            'GFDL-CM4','GFDL-ESM4','UKESM1-0-LL']

tas_mod_list = []

for mod in mod_list:
    print(mod)
    if mod not in ['GISS_E2-1-H']:
        tas_scen_list = []
        for scen in ['historical','ssp126','ssp245','ssp585']:
            if (mod == 'GFDL-CM4') and (scen == 'ssp126'):
                tas_mod_scen['tas'] = tas_mod_scen['tas']*np.nan
            else:
                tas_mod_scen = xr.open_dataset(inputpath_tas+mod+'/tas_Amon_'+mod+'_'+scen+'_fldmean_ymean.nc')
                #tas_mod_scen = xr.open_dataset(inputpath_tas+mod+'/tas_Amon_'+mod+'_'+scen+'_ymean_fldmeansouthof60.nc')
                tas_mod_scen['time'] = tas_mod_scen.time.dt.year
            tas_out = tas_mod_scen['tas'].squeeze(drop=True)
            if 'height' in tas_out.coords:
                tas_out = tas_out.drop('height')
            tas_scen_list.append(tas_out.assign_coords({'scenario':scen}))

        tas_all_scen = xr.concat(tas_scen_list,dim='scenario')
        tas_mod_list.append(tas_all_scen.assign_coords({'model':mod}))
        
tas_all_mod = xr.concat(tas_mod_list, dim='model') 

ACCESS-CM2
ACCESS-ESM1-5
CNRM-CM6-1
CNRM-ESM2-1
GISS_E2-1-H
CESM2-WACCM
CESM2
CanESM5
IPSL-CM6A-LR


/home/burgardc/miniforge3/envs/py38/lib/python3.8/site-packages/xarray/coding/times.py:716: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/home/burgardc/miniforge3/envs/py38/lib/python3.8/site-packages/xarray/coding/times.py:716: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/home/burgardc/miniforge3/envs/py38/lib/python3.8/site-packages/xarray/core/indexing.py:529: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  return np.asarray(array[self.key], dtype=None)
/home/burgardc/miniforge3/envs/py38/

MRI-ESM2-0


/home/burgardc/miniforge3/envs/py38/lib/python3.8/site-packages/xarray/coding/times.py:716: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/home/burgardc/miniforge3/envs/py38/lib/python3.8/site-packages/xarray/coding/times.py:716: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/home/burgardc/miniforge3/envs/py38/lib/python3.8/site-packages/xarray/core/indexing.py:529: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  return np.asarray(array[self.key], dtype=None)
/home/burgardc/miniforge3/envs/py38/

MPI-ESM1-2-HR
GFDL-CM4
GFDL-ESM4
UKESM1-0-LL


In [10]:
tas_all_mod_ano = tas_all_mod - tas_all_mod.sel(scenario='historical',time=range(1850,1901)).mean('time')

Hydrofracturing limits

In [11]:
hf_limit = xr.open_dataset(inputpath_data + 'hydrofracturing_limits_new.nc')['hydrofrac_limit']

PREPARE THE DATA

In [12]:
sorted_isf_rignot = [11,69,43,28,12,57,
                     70,44,29,13,58,71,45,30,14,
                     59,72,46,
                     31,
                     15,61,73,47,32,16,48,33,17,62,49,34,18,63,74,
                     50,35,19,64,
                     10,
                     36,20,65,51,37,
                     22,38,52,23,66,53,39,24,
                     67,40,54,75,25,41,
                     26,42,55,68,60,27]

In [13]:
sens_weights = xr.DataArray(data=np.array([0.11,
                                           0.24,
                                           0.03,
                                           0.10,
                                           0.10,
                                           0.10,
                                           0.10,
                                           0.24,
                                           0.47,
                                           0.41,
                                           0.12,
                                           0.43,
                                           0.39,
                                           0.05]), dims=['model']).assign_coords({'model': 
                                                                                  ['ACCESS-CM2','ACCESS-ESM1-5','CanESM5',
                                                                                   'CESM2','CESM2-WACCM','CNRM-CM6-1','CNRM-ESM2-1',
                                                                                   'GFDL-CM4','GFDL-ESM4','GISS-E2-1-H', 'IPSL-CM6A-LR',
                                                                                   'MPI-ESM1-2-HR','MRI-ESM2-0','UKESM1-0-LL']})
sens_weights = sens_weights.drop_sel(model='GISS-E2-1-H')

In [14]:
model_2300 = ['ACCESS-CM2','ACCESS-ESM1-5','CanESM5','CESM2-WACCM', 'IPSL-CM6A-LR','MRI-ESM2-0','UKESM1-0-LL'] #'GISS-E2-1-H',

In [15]:
bay_weights = weight_file['bay_weights']
bay_weights_2300 = weight_2300_file['bay_weights']

In [16]:
total_weights = bay_weights * sens_weights
total_weights_2300 = bay_weights_2300 * sens_weights

In [18]:
final_weights = xr.merge([total_weights.to_dataset(name='weights_2100_ensemble'), total_weights_2300.to_dataset(name='weights_2300_ensemble')]).drop('metrics').drop('box_nb_tot').drop('config')
final_weights.sum(['m','param']).to_netcdf(inputpath_weights + 'combined_weights_for_Nico.nc')

PermissionError: [Errno 13] Permission denied: b'/bettik/burgardc/DATA/SUMMER_PAPER/processed/ANALYSIS/combined_weights_for_Nico.nc'

In [17]:
# CALVING = 0
mass_balance_weighted_yy = (file_viability_info['MASS_BALANCE'] - file_viability_info['CALVING']).sel(time=range(1850,2101)).weighted(bay_weights * sens_weights)
mass_balance_weighted_yy_2300 = (file_viability_info['MASS_BALANCE'] - file_viability_info['CALVING']).sel(time=range(1850,2300)).sel(model=model_2300).weighted(bay_weights_2300 * sens_weights.sel(model=model_2300))

In [42]:
#mass_balance_weighted_yy = file_viability_info['MASS_BALANCE'].sel(time=range(1850,2101)).weighted(bay_weights * sens_weights)
#mass_balance_weighted_yy_2300 = file_viability_info['MASS_BALANCE'].sel(time=range(1850,2300)).sel(model=model_2300).weighted(bay_weights_2300 * sens_weights.sel(model=model_2300))

In [18]:
weighted_quantiles = mass_balance_weighted_yy.quantile([0,0.1,0.33,0.66,0.5,0.9,1],dim=['model','param','m'])
weighted_quantiles_2300 = mass_balance_weighted_yy_2300.quantile([0,0.1,0.33,0.66,0.5,0.9,1],dim=['model','param','m'])

In [19]:
unviable_times = weighted_quantiles > 0
unviable_times_2300 = weighted_quantiles_2300 > 0


In [20]:
viability_diff = unviable_times.sel(time=range(2100,1849,-1)).cumsum('time').diff('time')
limit_max = viability_diff.time.where(viability_diff == 0).max('time') + 1
limit_max = limit_max.where(limit_max<2099)
limit_max = limit_max.where(~(np.isnan(limit_max) & unviable_times.sel(time=1850)), 1850)

limit_max_full = limit_max.where(np.isfinite(limit_max), 2301)

count_unviable_isf = (unviable_times.time >= limit_max_full).sum('Nisf')

In [21]:
viability_diff_2300 = unviable_times_2300.sel(time=range(2298,1849,-1)).cumsum('time').diff('time')
limit_max_2300 = viability_diff_2300.time.where(viability_diff_2300 == 0).max('time') + 1
limit_max_2300 = limit_max_2300.where(limit_max_2300<2299)
limit_max_2300 = limit_max_2300.where(~(np.isnan(limit_max_2300) & unviable_times_2300.sel(time=1850)), 1850)

limit_max_full_2300 = limit_max_2300.where(np.isfinite(limit_max_2300), 2305)

count_unviable_isf_2300 = (unviable_times_2300.time >= limit_max_full_2300).sum('Nisf')

Do the same for hydrofracturing

In [26]:
limit_max_full_2300

<xarray.DataArray 'time' (quantile: 7, scenario: 3, Nisf: 64)>
array([[[2298., 2298., 2298., ..., 2298., 2298., 2298.],
        [2298., 2298., 2298., ..., 2298., 2298., 2298.],
        [2194., 2298., 2288., ..., 2169., 2298., 2163.]],

       [[2298., 2298., 2298., ..., 2298., 2298., 2298.],
        [2298., 2298., 2298., ..., 2298., 2298., 2298.],
        [2175., 2298., 2187., ..., 2129., 2298., 2100.]],

       [[2298., 2298., 2298., ..., 2298., 2298., 2298.],
        [2298., 2298., 2298., ..., 2298., 2298., 2298.],
        [2137., 2298., 2121., ..., 2104., 2298., 2084.]],

       ...,

       [[2298., 2298., 2298., ..., 2298., 2298., 2298.],
        [2298., 2298., 2298., ..., 2298., 2298., 2298.],
        [2133., 2229., 2111., ..., 2091., 2298., 2061.]],

       [[2298., 2298., 2298., ..., 2055., 2298., 1989.],
        [2298., 2298., 2298., ..., 2298., 2298., 2298.],
        [2100., 2137., 2070., ..., 2048., 2293., 1989.]],

       [[2298., 2298., 2028., ..., 2006., 2298., 1850.],
        [2298., 2298., 2298., ..., 2298., 2298., 2298.],
        [2091., 2090., 2010., ..., 2006., 2128., 1850.]]])
Coordinates:
    metrics     <U2 'Gt'
    box_nb_tot  int64 5
    config      int64 2
  * Nisf        (Nisf) int64 10 11 12 13 14 15 16 17 ... 68 69 70 71 72 73 74 75
  * scenario    (scenario) object 'ssp126' 'ssp245' 'ssp585'
    ID_IMBIE    (Nisf) int64 66 124 132 5 13 22 31 38 ... 127 1 8 18 26 55 109
  * quantile    (quantile) float64 0.0 0.1 0.33 0.66 0.5 0.9 1.0
    time        int64 1850

In [22]:
count_hflimit_isf_2300 = (unviable_times_2300.time >= hf_limit).sum('Nisf')

Try to find some limit

In [33]:
limit_of_int = limit_max_full_2300.sel(scenario='ssp585',quantile=0.33)
time_of_int = weighted_mean_tas.sel(scenario='ssp585').time.where((weighted_mean_tas.sel(scenario='ssp585') > 4) & (weighted_mean_tas.sel(scenario='ssp585') <= 5), drop=True)
isf_4_5 = limit_of_int.where((limit_of_int >= time_of_int.min()) & (limit_of_int <= time_of_int.max()), drop=True)

NameError: name 'limit_max_full_2300' is not defined

In [ ]:
time_of_int = weighted_mean_tas.sel(scenario='ssp585').time.where((weighted_mean_tas.sel(scenario='ssp585') > 5) & (weighted_mean_tas.sel(scenario='ssp585') <= 6), drop=True)
isf_5_6 = limit_of_int.where((limit_of_int >= time_of_int.min()) & (limit_of_int <= time_of_int.max()), drop=True)

In [ ]:
time_of_int = weighted_mean_tas.sel(scenario='ssp585').time.where((weighted_mean_tas.sel(scenario='ssp585') > 6) & (weighted_mean_tas.sel(scenario='ssp585') <= 7), drop=True)
isf_6_7 = limit_of_int.where((limit_of_int >= time_of_int.min()) & (limit_of_int <= time_of_int.max()), drop=True)

In [ ]:
for kisf in isf_4_5.Nisf:
    print(file_isf['isf_name'].sel(Nisf=kisf).values)

In [ ]:
for kisf in isf_5_6.Nisf:
    print(file_isf['isf_name'].sel(Nisf=kisf).values)

In [ ]:
for kisf in isf_6_7.Nisf:
    print(file_isf['isf_name'].sel(Nisf=kisf).values)

In [ ]:
(weighted_mean_tas.sel(scenario='ssp585') > 4).time.where((weighted_mean_tas.sel(scenario='ssp585') > 4) & (weighted_mean_tas.sel(scenario='ssp585') <= 5), drop=True)

In [23]:
weighted_tas_2100 = tas_all_mod_ano.weighted(bay_weights * sens_weights) 
weighted_mean_tas_2100 = weighted_tas_2100.mean(['Nisf','model','param','m'])
weighted_tas_std_2100 = weighted_tas_2100.std(['Nisf','model','param','m'])


In [24]:
weighted_tas = tas_all_mod_ano.sel(model=model_2300).weighted(bay_weights_2300 * sens_weights.sel(model=model_2300)) 
weighted_mean_tas = weighted_tas.mean(['Nisf','model','param','m'])
weighted_tas_std = weighted_tas.std(['Nisf','model','param','m'])


In [25]:
scen = 'ssp126'
scen2 = 'ssp585'
scen3 = 'ssp245'

f = plt.figure()
f.set_size_inches(8.25/1.5, 8.25/1.5)

weighted_mean_tas_2100.sel(scenario=scen,time=range(1850,2101)).plot()
plt.fill_between(x=range(1850,2101),y1=weighted_mean_tas_2100.sel(scenario=scen,time=range(1850,2101)) - weighted_tas_std_2100.sel(scenario=scen,time=range(1850,2101)), 
                 y2=weighted_mean_tas_2100.sel(scenario=scen,time=range(1850,2101)) + weighted_tas_std_2100.sel(scenario=scen,time=range(1850,2101)), alpha=0.5)

weighted_mean_tas_2100.sel(scenario=scen3,time=range(1850,2101)).plot()
plt.fill_between(x=range(1850,2101),y1=weighted_mean_tas_2100.sel(scenario=scen3,time=range(1850,2101)) - weighted_tas_std_2100.sel(scenario=scen3,time=range(1850,2101)), 
                 y2=weighted_mean_tas_2100.sel(scenario=scen3,time=range(1850,2101)) + weighted_tas_std_2100.sel(scenario=scen3,time=range(1850,2101)), alpha=0.5)

weighted_mean_tas_2100.sel(scenario=scen2,time=range(1850,2101)).plot()
plt.fill_between(x=range(1850,2101),y1=weighted_mean_tas_2100.sel(scenario=scen2,time=range(1850,2101)) - weighted_tas_std_2100.sel(scenario=scen2,time=range(1850,2101)), 
                 y2=weighted_mean_tas_2100.sel(scenario=scen2,time=range(1850,2101)) + weighted_tas_std_2100.sel(scenario=scen2,time=range(1850,2101)), alpha=0.5)

sns.despine()
plt.title('')
plt.xlabel('Time')
plt.ylabel('Global surface air temperature anomaly w.r.t. 1850-1900 [K]')
f.savefig(plot_path + 'tas_evolution_2100.png',dpi=250)
#plt.axvline(x=2075,linestyle='--')
#plt.axvline(x=2180,linestyle='--')

libGL error: No matching fbConfigs or visuals found
libGL error: failed to load driver: swrast


In [50]:
scen = 'ssp126'
scen2 = 'ssp585'
scen3 = 'ssp245'

f = plt.figure()
f.set_size_inches(8.25/1.5, 8.25/1.5)

weighted_mean_tas.sel(scenario=scen).plot(color='pink')
plt.fill_between(x=range(1850,2301),y1=weighted_mean_tas.sel(scenario=scen) - weighted_tas_std.sel(scenario=scen), 
                 y2=weighted_mean_tas.sel(scenario=scen) + weighted_tas_std.sel(scenario=scen), alpha=0.5, color='pink')

weighted_mean_tas.sel(scenario=scen3).plot(color='orchid')
plt.fill_between(x=range(1850,2301),y1=weighted_mean_tas.sel(scenario=scen3) - weighted_tas_std.sel(scenario=scen3), 
                 y2=weighted_mean_tas.sel(scenario=scen3) + weighted_tas_std.sel(scenario=scen3), alpha=0.5, color='orchid')

weighted_mean_tas.sel(scenario=scen2).plot(color='darkmagenta')
plt.fill_between(x=range(1850,2301),y1=weighted_mean_tas.sel(scenario=scen2) - weighted_tas_std.sel(scenario=scen2), 
                 y2=weighted_mean_tas.sel(scenario=scen2) + weighted_tas_std.sel(scenario=scen2), alpha=0.5, color='darkmagenta')

plt.axhline(y=4.7,xmin=(2085-2014)/(2300-2014),xmax=1,color='k',linestyle='--')
#plt.axhline(y=9.8,xmin=(2185-2014)/(2300-2014),xmax=1,color='k',linestyle='--')
plt.axvline(x=2085,ymax=4.7/14.5,color='k',linestyle='--')
#plt.axvline(x=2185,ymax=9.8/14.5,color='k',linestyle='--')


#plt.text(2200,4.7,'4.7$\pm$0.6')

sns.despine()
plt.title('')
plt.xlim(2014,2300)
plt.ylim(0,14.5)
plt.xlabel('Time')
plt.ylabel('Global surface air temperature anomaly w.r.t. 1850-1900 [K]')
f.savefig(plot_path + 'tas_evolution.png',dpi=250)
#f.savefig(plot_path + 'tas_evolution_southof60.png',dpi=250)


#plt.axvline(x=2075,linestyle='--')
#plt.axvline(x=2180,linestyle='--')

In [51]:
scen = 'ssp126'
scen2 = 'ssp585'
scen3 = 'ssp245'

f = plt.figure()
f.set_size_inches(8.25/1.5, 8.25/1.5)

plt.scatter(count_unviable_isf_2300.sel(scenario=scen2,quantile=0.33,time=range(1850,2298)), weighted_mean_tas.sel(scenario=scen2,time=range(1850,2298)),s=10,alpha=0.3,color='darkmagenta')
plt.scatter(count_unviable_isf_2300.sel(scenario=scen3,quantile=0.33,time=range(1850,2298)), weighted_mean_tas.sel(scenario=scen3,time=range(1850,2298)),s=10,alpha=0.3,color='orchid')
plt.scatter(count_unviable_isf_2300.sel(scenario=scen,quantile=0.33,time=range(1850,2298)), weighted_mean_tas.sel(scenario=scen,time=range(1850,2298)),s=10,alpha=0.3,color='pink')

plt.axhline(y=4.7,color='k',linestyle='--')
#plt.axhline(y=9.8,color='k',linestyle='--')
plt.text(20,4.9,'4.7$\pm$0.6°C')
#plt.text(20,10,'9.8$\pm$1.8°C')

sns.despine()
plt.title('')
plt.ylim(0,14.5)
plt.ylabel('Global surface air temperature anomaly w.r.t. 1850-1900 [K]')
plt.xlabel('Number of likely non-viable ice shelves')
#plt.axhline(y=4.5)
f.savefig(plot_path + 'tas_nonviable_scatter.png',dpi=250)
#f.savefig(plot_path + 'tas_southof60_nonviable_scatter.png',dpi=250)

In [28]:
scen = 'ssp126'
scen2 = 'ssp585'
scen3 = 'ssp245'

f = plt.figure()
f.set_size_inches(8.25/1.5, 8.25/1.5)

plt.scatter(count_unviable_isf_2300.sel(scenario=scen,quantile=0.33,time=range(1850,2298)), weighted_mean_tas.sel(scenario=scen,time=range(1850,2298)),s=10,alpha=0.3)
plt.scatter(count_unviable_isf_2300.sel(scenario=scen3,quantile=0.33,time=range(1850,2298)), weighted_mean_tas.sel(scenario=scen3,time=range(1850,2298)),s=10,alpha=0.3)
plt.scatter(count_unviable_isf_2300.sel(scenario=scen2,quantile=0.33,time=range(1850,2298)), weighted_mean_tas.sel(scenario=scen2,time=range(1850,2298)),s=10,alpha=0.3)


sns.despine()
plt.title('')
plt.ylim(0,14.5)
plt.ylabel('Global surface air temperature anomaly w.r.t. 1850-1900 [K]')
plt.xlabel('Number of likely non-viable ice shelves')
f.savefig(plot_path + 'tas_nonviable_scatter_calving0.png',dpi=250)

In [29]:
scen = 'ssp126'
scen2 = 'ssp585'
scen3 = 'ssp245'

f = plt.figure()
f.set_size_inches(8.25/1.5, 8.25/1.5)

plt.scatter(weighted_mean_tas_2100.sel(scenario=scen,time=range(1850,2101)),count_unviable_isf.sel(scenario=scen,quantile=0.33,time=range(1850,2101)),s=10,alpha=0.3)
plt.scatter(weighted_mean_tas_2100.sel(scenario=scen3,time=range(1850,2101)),count_unviable_isf.sel(scenario=scen3,quantile=0.33,time=range(1850,2101)),s=10,alpha=0.3)
plt.scatter(weighted_mean_tas_2100.sel(scenario=scen2,time=range(1850,2101)),count_unviable_isf.sel(scenario=scen2,quantile=0.33,time=range(1850,2101)),s=10,alpha=0.3)


sns.despine()
plt.title('')
plt.xlabel('Global surface air temperature anomaly w.r.t. 1850-1900 [K]')
plt.ylabel('Number of likely non-viable ice shelves')
f.savefig(plot_path + 'tas_nonviable_scatter_2100.png',dpi=250)

In [ ]:
weighted_mean_tas.sel(scenario=scen2,time=2300)

In [ ]:
weighted_mean_tas.sel(scenario='ssp585',time=2185)

<xarray.DataArray 'tas' ()>
array(9.88549744)
Coordinates:
    time        int64 2185
    scenario    <U10 'ssp585'
    metrics     <U2 'Gt'
    box_nb_tot  int64 5
    config      int64 2

In [ ]:
weighted_tas_std.sel(scenario='ssp585',time=2085)

<xarray.DataArray 'tas' ()>
array(0.63072946)
Coordinates:
    time        int64 2085
    scenario    <U10 'ssp585'
    metrics     <U2 'Gt'
    box_nb_tot  int64 5
    config      int64 2

In [ ]:
weighted_mean_tas.sel(scenario=scen2,time=2300)

In [31]:
file_isf['isf_name'].sel(Nisf=24).load()

<xarray.DataArray 'isf_name' ()>
array('Venable', dtype='<U7')
Coordinates:
    Nisf     int64 24
Attributes:
    standard_name:  ice shelf name

In [51]:
count_unviable_isf_2300.sel(time=2255, scenario='ssp126', quantile=0.9)

<xarray.DataArray 'time' ()>
array(16)
Coordinates:
    time        int64 2255
    metrics     <U2 'Gt'
    box_nb_tot  int64 5
    config      int64 2
    scenario    <U6 'ssp126'
    quantile    float64 0.9

In [32]:

limit_max_full_2300.sel(quantile=0.33,scenario='ssp126').where(limit_max_full_2300.sel(quantile=0.33,scenario='ssp126') < 2298).dropna('Nisf')

<xarray.DataArray 'time' (Nisf: 2)>
array([2293., 2297.])
Coordinates:
    metrics     <U2 'Gt'
    box_nb_tot  int64 5
    config      int64 2
  * Nisf        (Nisf) int64 24 25
    scenario    <U6 'ssp126'
    ID_IMBIE    (Nisf) int64 92 99
    quantile    float64 0.33
    time        int64 1850

In [112]:
date_of_int = 2084
limit_max_full_2300.sel(quantile=0.33,scenario='ssp585').where(limit_max_full_2300.sel(quantile=0.33,scenario='ssp585') < date_of_int).dropna('Nisf')

<xarray.DataArray 'time' (Nisf: 2)>
array([2040., 2024.])
Coordinates:
    metrics     <U2 'Gt'
    box_nb_tot  int64 5
    config      int64 2
  * Nisf        (Nisf) int64 24 25
    scenario    <U6 'ssp585'
    ID_IMBIE    (Nisf) int64 92 99
    quantile    float64 0.33
    time        int64 1850

In [111]:
date_of_int = 2184
limit_max_full_2300.sel(quantile=0.33,scenario='ssp585').where(limit_max_full_2300.sel(quantile=0.33,scenario='ssp585') < date_of_int).dropna('Nisf')

<xarray.DataArray 'time' (Nisf: 34)>
array([2137., 2121., 2116., 2138., 2099., 2135., 2040., 2024., 2138.,
       2117., 2122., 2117., 2088., 2164., 2094., 2092., 2144., 2118.,
       2132., 2099., 2087., 2104., 2111., 2105., 2111., 2100., 2129.,
       2129., 2165., 2097., 2132., 2103., 2104., 2084.])
Coordinates:
    metrics     <U2 'Gt'
    box_nb_tot  int64 5
    config      int64 2
  * Nisf        (Nisf) int64 10 12 13 17 18 22 24 25 ... 61 65 66 69 70 71 73 75
    scenario    <U6 'ssp585'
    ID_IMBIE    (Nisf) int64 66 132 5 38 47 82 92 99 ... 24 75 86 127 1 8 26 109
    quantile    float64 0.33
    time        int64 1850

In [90]:
26/64

0.40625

In [ ]:
(35-3)/64

In [ ]:
diff_scen = ( (unviable_times.time >= limit_max_full).sel(scenario='ssp126').sel(time=range(1850,2000)).astype(int) - (unviable_times.time >= limit_max_full).sel(scenario='ssp585').sel(time=range(1850,2000)).astype(int))

In [ ]:
unviable_times.sel(Nisf=25,quantile=0.33,time=1850)

In [ ]:
limit_max_full.sel(Nisf=42,quantile=0.9)

In [ ]:
diff_scen.where(diff_scen != 0, drop=True).sel(quantile=0.9,Nisf=42)

In [31]:

### THINK ABOUT THE SIGMA_MOD AND SIGMA_OBS AGAIN

regions = ['Weddell','Bellingshausen','Amundsen','Ross','East 1','East 2','Dronning Maud Land']
colors = ['red','orange','gold','mediumturquoise','maroon','magenta','cornflowerblue','lightseagreen',
          'yellowgreen','royalblue','tomato','darkslateblue','purple','darkgoldenrod']

f = plt.figure()
f.set_size_inches(8.25*3, 8.25)

ax={}

leg_hdl = []

i = 0

plt.fill_between(x=range(1850,2014), y1=count_unviable_isf.sel(quantile=0.33,scenario='ssp585',time=range(1850,2014)), y2=count_unviable_isf.sel(quantile=0.66,scenario='ssp585',time=range(1850,2014)), color='grey',alpha=0.2)
plt.fill_between(x=range(2014,2101), y1=count_unviable_isf.sel(quantile=0.33,scenario='ssp585',time=range(2014,2101)), y2=count_unviable_isf.sel(quantile=0.66,scenario='ssp585',time=range(2014,2101)), color='deeppink',alpha=0.2)
plt.fill_between(x=range(2014,2101), y1=count_unviable_isf.sel(quantile=0.33,scenario='ssp245',time=range(2014,2101)), y2=count_unviable_isf.sel(quantile=0.66,scenario='ssp245',time=range(2014,2101)), color='orange',alpha=0.2)
plt.fill_between(x=range(2014,2101), y1=count_unviable_isf.sel(quantile=0.33,scenario='ssp126',time=range(2014,2101)), y2=count_unviable_isf.sel(quantile=0.66,scenario='ssp126',time=range(2014,2101)), color='violet',alpha=0.2)

plt.plot(range(1850,2014),count_unviable_isf.sel(quantile=0.5,scenario='ssp585',time=range(1850,2014)), color='k')
plt.plot(range(2014,2101),count_unviable_isf.sel(quantile=0.5,scenario='ssp585',time=range(2014,2101)), color='deeppink')
plt.plot(range(2014,2101),count_unviable_isf.sel(quantile=0.5,scenario='ssp245',time=range(2014,2101)), color='orange')
plt.plot(range(2014,2101),count_unviable_isf.sel(quantile=0.5,scenario='ssp126',time=range(2014,2101)), color='violet')

plt.plot(range(1850,2014),count_unviable_isf.sel(quantile=0.1,scenario='ssp585',time=range(1850,2014)), color='k', linestyle = '--')
plt.plot(range(2014,2101),count_unviable_isf.sel(quantile=0.1,scenario='ssp585',time=range(2014,2101)), color='deeppink', linestyle = '--')
plt.plot(range(2014,2101),count_unviable_isf.sel(quantile=0.1,scenario='ssp245',time=range(2014,2101)), color='orange', linestyle = '--')
plt.plot(range(2014,2101),count_unviable_isf.sel(quantile=0.1,scenario='ssp126',time=range(2014,2101)), color='violet', linestyle = '--')

plt.plot(range(1850,2014),count_unviable_isf.sel(quantile=0.9,scenario='ssp585',time=range(1850,2014)), color='k', linestyle = '--')
plt.plot(range(2014,2101),count_unviable_isf.sel(quantile=0.9,scenario='ssp585',time=range(2014,2101)), color='red', linestyle = '--')
plt.plot(range(2014,2101),count_unviable_isf.sel(quantile=0.9,scenario='ssp245',time=range(2014,2101)), color='orange', linestyle = '--')
plt.plot(range(2014,2101),count_unviable_isf.sel(quantile=0.9,scenario='ssp126',time=range(2014,2101)), color='violet', linestyle = '--')

#f.legend()
#f.subplots_adjust(bottom=0.05, wspace=0.1)

f.tight_layout()
sns.despine()
plt.savefig(plot_path + 'nb_non_viable_isf_2100_withoutGISS.pdf')

In [ ]:

### THINK ABOUT THE SIGMA_MOD AND SIGMA_OBS AGAIN

regions = ['Weddell','Bellingshausen','Amundsen','Ross','East 1','East 2','Dronning Maud Land']
colors = ['red','orange','gold','mediumturquoise','maroon','magenta','cornflowerblue','lightseagreen',
          'yellowgreen','royalblue','tomato','darkslateblue','purple','darkgoldenrod']

f = plt.figure()
#f.set_size_inches(8.25, 8.25)
f.set_size_inches(8.25*3, 8.25)

ax={}

leg_hdl = []

i = 0

plt.fill_between(x=range(1850,2014), y1=count_unviable_isf_2300.sel(quantile=0.33,scenario='ssp585',time=range(1850,2014)), y2=count_unviable_isf_2300.sel(quantile=0.66,scenario='ssp585',time=range(1850,2014)), color='grey',alpha=0.2)
plt.fill_between(x=range(2014,2299), y1=count_unviable_isf_2300.sel(quantile=0.33,scenario='ssp585',time=range(2014,2299)), y2=count_unviable_isf_2300.sel(quantile=0.66,scenario='ssp585',time=range(2014,2299)), color='deeppink',alpha=0.2)
plt.fill_between(x=range(2014,2299), y1=count_unviable_isf_2300.sel(quantile=0.33,scenario='ssp126',time=range(2014,2299)), y2=count_unviable_isf_2300.sel(quantile=0.66,scenario='ssp126',time=range(2014,2299)), color='violet',alpha=0.2)

plt.plot(range(1850,2014),count_unviable_isf_2300.sel(quantile=0.5,scenario='ssp585',time=range(1850,2014)), color='k')
plt.plot(range(2014,2299),count_unviable_isf_2300.sel(quantile=0.5,scenario='ssp585',time=range(2014,2299)), color='deeppink')
plt.plot(range(2014,2299),count_unviable_isf_2300.sel(quantile=0.5,scenario='ssp126',time=range(2014,2299)), color='violet')

plt.plot(range(1850,2014),count_unviable_isf_2300.sel(quantile=0.1,scenario='ssp585',time=range(1850,2014)), color='k', linestyle = '--')
plt.plot(range(2014,2299),count_unviable_isf_2300.sel(quantile=0.1,scenario='ssp585',time=range(2014,2299)), color='deeppink', linestyle = '--')
plt.plot(range(2014,2299),count_unviable_isf_2300.sel(quantile=0.1,scenario='ssp126',time=range(2014,2299)), color='violet', linestyle = '--')

plt.plot(range(1850,2014),count_unviable_isf_2300.sel(quantile=0.9,scenario='ssp585',time=range(1850,2014)), color='k', linestyle = '--')
plt.plot(range(2014,2299),count_unviable_isf_2300.sel(quantile=0.9,scenario='ssp585',time=range(2014,2299)), color='deeppink', linestyle = '--')
plt.plot(range(2014,2299),count_unviable_isf_2300.sel(quantile=0.9,scenario='ssp126',time=range(2014,2299)), color='violet', linestyle = '--')

#f.legend()
#f.subplots_adjust(bottom=0.05, wspace=0.1)
plt.xlim(1850,2298)
plt.ylabel('Number of non-viable ice shelves')
plt.xlabel('Time')
f.tight_layout()
sns.despine()
plt.savefig(plot_path + 'nb_non_viable_isf_2300.pdf')

In [ ]:

### THINK ABOUT THE SIGMA_MOD AND SIGMA_OBS AGAIN

regions = ['Weddell','Bellingshausen','Amundsen','Ross','East 1','East 2','Dronning Maud Land']
colors = ['red','orange','gold','mediumturquoise','maroon','magenta','cornflowerblue','lightseagreen',
          'yellowgreen','royalblue','tomato','darkslateblue','purple','darkgoldenrod']

f = plt.figure()
#f.set_size_inches(8.25, 8.25)
f.set_size_inches(8.25*1.5, 8.25/3*1.5)

ax={}

leg_hdl = []

i = 0

#plt.fill_between(x=range(1850,2014), y1=count_unviable_isf_2300.sel(quantile=0.33,scenario='ssp585',time=range(1850,2014)), y2=count_unviable_isf_2300.sel(quantile=0.66,scenario='ssp585',time=range(1850,2014)), color='grey',alpha=0.2)
#plt.fill_between(x=range(2014,2299), y1=count_unviable_isf_2300.sel(quantile=0.33,scenario='ssp585',time=range(2014,2299)), y2=count_unviable_isf_2300.sel(quantile=0.66,scenario='ssp585',time=range(2014,2299)), color='deeppink',alpha=0.2)
#plt.fill_between(x=range(2014,2299), y1=count_unviable_isf_2300.sel(quantile=0.33,scenario='ssp126',time=range(2014,2299)), y2=count_unviable_isf_2300.sel(quantile=0.66,scenario='ssp126',time=range(2014,2299)), color='violet',alpha=0.2)

for qq in [0.1,0.33,0.66,0.9]: #
    plt.plot(range(1850,2014),count_unviable_isf_2300.sel(quantile=qq,scenario='ssp585',time=range(1850,2014)), color=cmap(1-qq), linestyle='dashdot',linewidth=2)
    plt.plot(range(2014,2298),count_unviable_isf_2300.sel(quantile=qq,scenario='ssp585',time=range(2014,2298)), color=cmap(1-qq), linestyle='-',linewidth=2)
    plt.plot(range(2014,2298),count_unviable_isf_2300.sel(quantile=qq,scenario='ssp126',time=range(2014,2298)), color=cmap(1-qq), linestyle='--',linewidth=2)

#plt.plot(range(1850,2014),count_unviable_isf_2300.sel(quantile=0.1,scenario='ssp585',time=range(1850,2014)), color='k', linestyle = '--')
#plt.plot(range(2014,2299),count_unviable_isf_2300.sel(quantile=0.1,scenario='ssp585',time=range(2014,2299)), color='deeppink', linestyle = '--')
#plt.plot(range(2014,2299),count_unviable_isf_2300.sel(quantile=0.1,scenario='ssp126',time=range(2014,2299)), color='violet', linestyle = '--')

#plt.plot(range(1850,2014),count_unviable_isf_2300.sel(quantile=0.9,scenario='ssp585',time=range(1850,2014)), color='k', linestyle = '--')
#plt.plot(range(2014,2299),count_unviable_isf_2300.sel(quantile=0.9,scenario='ssp585',time=range(2014,2299)), color='deeppink', linestyle = '--')
#plt.plot(range(2014,2299),count_unviable_isf_2300.sel(quantile=0.9,scenario='ssp126',time=range(2014,2299)), color='violet', linestyle = '--')

#f.legend()
#f.subplots_adjust(bottom=0.05, wspace=0.1)
plt.xlim(1850,2298)
plt.ylabel('Number of non-viable ice shelves')
plt.xlabel('Time')
f.tight_layout()
sns.despine()
plt.savefig(plot_path + 'nb_non_viable_isf_2300_lines.pdf')

In [ ]:
count_unviable_isf_2300.sel(quantile=qq_list[n-1],scenario=scen,time=range(1850,2014))

In [43]:
scen='ssp126'

### THINK ABOUT THE SIGMA_MOD AND SIGMA_OBS AGAIN

regions = ['Weddell','Bellingshausen','Amundsen','Ross','East 1','East 2','Dronning Maud Land']
colors = ['red','orange','gold','mediumturquoise','maroon','magenta','cornflowerblue','lightseagreen',
          'yellowgreen','royalblue','tomato','darkslateblue','purple','darkgoldenrod']

f = plt.figure()
#f.set_size_inches(8.25, 8.25)
f.set_size_inches(8.25*0.75, 8.25/2*0.75) #*1.25

ax={}

leg_hdl = []

i = 0

#plt.fill_between(x=range(1850,2014), y1=count_unviable_isf_2300.sel(quantile=0.33,scenario='ssp585',time=range(1850,2014)), y2=count_unviable_isf_2300.sel(quantile=0.66,scenario='ssp585',time=range(1850,2014)), color='grey',alpha=0.2)
#plt.fill_between(x=range(2014,2299), y1=count_unviable_isf_2300.sel(quantile=0.33,scenario='ssp585',time=range(2014,2299)), y2=count_unviable_isf_2300.sel(quantile=0.66,scenario='ssp585',time=range(2014,2299)), color='deeppink',alpha=0.2)
#plt.fill_between(x=range(2014,2299), y1=count_unviable_isf_2300.sel(quantile=0.33,scenario='ssp126',time=range(2014,2299)), y2=count_unviable_isf_2300.sel(quantile=0.66,scenario='ssp126',time=range(2014,2299)), color='violet',alpha=0.2)

#qq_list = [1,0.9,0.66,0.33,0.1,0]
qq_list = [0,0.1,0.33,0.66,0.9]
col_tot = 4

for n in range(len(qq_list)-1,0,-1):
    if qq_list[n] == 0.1:
        plt.fill_between(x=range(1850,2015),y1=0,
                                    y2=count_unviable_isf_2300.sel(quantile=qq_list[n],scenario=scen,time=range(1850,2015)), color=cmap_new2(1-0.00001))
        plt.fill_between(x=range(2014,2298),y1=0,
                                            y2=count_unviable_isf_2300.sel(quantile=qq_list[n],scenario=scen,time=range(2014,2298)), color=cmap_new2(1-0.0001))
        plt.fill_between(x=range(1850,2015),y1=count_unviable_isf_2300.sel(quantile=qq_list[n],scenario=scen,time=range(1850,2015)),
                                            y2=count_unviable_isf_2300.sel(quantile=qq_list[n+1],scenario=scen,time=range(1850,2015)), color=cmap_new2(1 - n/col_tot))
        plt.fill_between(x=range(2014,2298),y1=count_unviable_isf_2300.sel(quantile=qq_list[n],scenario=scen,time=range(2014,2298)),
                                            y2=count_unviable_isf_2300.sel(quantile=qq_list[n+1],scenario=scen,time=range(2014,2298)), color=cmap_new2(1 - n/col_tot))
    elif qq_list[n] == 0.9:
        plt.fill_between(x=range(1850,2015),y1=count_unviable_isf_2300.sel(quantile=qq_list[n],scenario=scen,time=range(1850,2015)), color=cmap_new2(1-n/col_tot),
                                            y2=64)
        plt.fill_between(x=range(2014,2298),y1=count_unviable_isf_2300.sel(quantile=qq_list[n],scenario=scen,time=range(2014,2298)), color=cmap_new2(1-n/col_tot),
                                            y2=64)
        
    else:
        plt.fill_between(x=range(1850,2015),y1=count_unviable_isf_2300.sel(quantile=qq_list[n],scenario=scen,time=range(1850,2015)),
                                            y2=count_unviable_isf_2300.sel(quantile=qq_list[n+1],scenario=scen,time=range(1850,2015)), color=cmap_new2(1 - n/col_tot))
        plt.fill_between(x=range(2014,2298),y1=count_unviable_isf_2300.sel(quantile=qq_list[n],scenario=scen,time=range(2014,2298)),
                                            y2=count_unviable_isf_2300.sel(quantile=qq_list[n+1],scenario=scen,time=range(2014,2298)), color=cmap_new2(1 - n/col_tot))

#f.legend()
#f.subplots_adjust(bottom=0.05, wspace=0.1)
plt.xlim(1850,2298)
plt.ylim(0,64)
plt.ylabel('Number of non-viable ice shelves')
plt.xlabel('Time')
f.tight_layout()
sns.despine()
plt.savefig(plot_path + 'nb_non_viable_isf_2300_withoutGISS_fill_'+scen+'.pdf')

In [26]:
scen='ssp126'

### THINK ABOUT THE SIGMA_MOD AND SIGMA_OBS AGAIN

regions = ['Weddell','Bellingshausen','Amundsen','Ross','East 1','East 2','Dronning Maud Land']
colors = ['red','orange','gold','mediumturquoise','maroon','magenta','cornflowerblue','lightseagreen',
          'yellowgreen','royalblue','tomato','darkslateblue','purple','darkgoldenrod']

f = plt.figure()
#f.set_size_inches(8.25, 8.25)
f.set_size_inches(8.25*0.75, 8.25/2*0.75) #*1.25

ax={}

leg_hdl = []

i = 0

#plt.fill_between(x=range(1850,2014), y1=count_unviable_isf_2300.sel(quantile=0.33,scenario='ssp585',time=range(1850,2014)), y2=count_unviable_isf_2300.sel(quantile=0.66,scenario='ssp585',time=range(1850,2014)), color='grey',alpha=0.2)
#plt.fill_between(x=range(2014,2299), y1=count_unviable_isf_2300.sel(quantile=0.33,scenario='ssp585',time=range(2014,2299)), y2=count_unviable_isf_2300.sel(quantile=0.66,scenario='ssp585',time=range(2014,2299)), color='deeppink',alpha=0.2)
#plt.fill_between(x=range(2014,2299), y1=count_unviable_isf_2300.sel(quantile=0.33,scenario='ssp126',time=range(2014,2299)), y2=count_unviable_isf_2300.sel(quantile=0.66,scenario='ssp126',time=range(2014,2299)), color='violet',alpha=0.2)

#qq_list = [1,0.9,0.66,0.33,0.1,0]
qq_list = [0,0.1,0.33,0.66,0.9]
col_tot = 4

for n in range(len(qq_list)-1,0,-1):
    if qq_list[n] == 0.1:
        plt.fill_between(x=range(1850,2015),y1=0,
                                    y2=count_unviable_isf_2300.sel(quantile=qq_list[n],scenario=scen,time=range(1850,2015)), color=cmap_new2(1-0.00001))
        plt.fill_between(x=range(2014,2298),y1=0,
                                            y2=count_unviable_isf_2300.sel(quantile=qq_list[n],scenario=scen,time=range(2014,2298)), color=cmap_new2(1-0.0001))
        plt.fill_between(x=range(1850,2015),y1=count_unviable_isf_2300.sel(quantile=qq_list[n],scenario=scen,time=range(1850,2015)),
                                            y2=count_unviable_isf_2300.sel(quantile=qq_list[n+1],scenario=scen,time=range(1850,2015)), color=cmap_new2(1 - n/col_tot))
        plt.fill_between(x=range(2014,2298),y1=count_unviable_isf_2300.sel(quantile=qq_list[n],scenario=scen,time=range(2014,2298)),
                                            y2=count_unviable_isf_2300.sel(quantile=qq_list[n+1],scenario=scen,time=range(2014,2298)), color=cmap_new2(1 - n/col_tot))
    elif qq_list[n] == 0.9:
        plt.fill_between(x=range(1850,2015),y1=count_unviable_isf_2300.sel(quantile=qq_list[n],scenario=scen,time=range(1850,2015)), color=cmap_new2(1-n/col_tot),
                                            y2=64)
        plt.fill_between(x=range(2014,2298),y1=count_unviable_isf_2300.sel(quantile=qq_list[n],scenario=scen,time=range(2014,2298)), color=cmap_new2(1-n/col_tot),
                                            y2=64)
        
    else:
        plt.fill_between(x=range(1850,2015),y1=count_unviable_isf_2300.sel(quantile=qq_list[n],scenario=scen,time=range(1850,2015)),
                                            y2=count_unviable_isf_2300.sel(quantile=qq_list[n+1],scenario=scen,time=range(1850,2015)), color=cmap_new2(1 - n/col_tot))
        plt.fill_between(x=range(2014,2298),y1=count_unviable_isf_2300.sel(quantile=qq_list[n],scenario=scen,time=range(2014,2298)),
                                            y2=count_unviable_isf_2300.sel(quantile=qq_list[n+1],scenario=scen,time=range(2014,2298)), color=cmap_new2(1 - n/col_tot))

#f.legend()
#f.subplots_adjust(bottom=0.05, wspace=0.1)
plt.xlim(1850,2298)
plt.ylim(0,64)
plt.ylabel('Number of non-viable ice shelves')
plt.xlabel('Time')
f.tight_layout()
sns.despine()
plt.savefig(plot_path + 'nb_non_viable_isf_2300_withoutGISS_fill_'+scen+'_calving0.pdf')

In [31]:
scen='ssp126'

### THINK ABOUT THE SIGMA_MOD AND SIGMA_OBS AGAIN

regions = ['Weddell','Bellingshausen','Amundsen','Ross','East 1','East 2','Dronning Maud Land']
colors = ['red','orange','gold','mediumturquoise','maroon','magenta','cornflowerblue','lightseagreen',
          'yellowgreen','royalblue','tomato','darkslateblue','purple','darkgoldenrod']

f = plt.figure()
#f.set_size_inches(8.25, 8.25)
f.set_size_inches(8.25*0.75, 8.25/2*0.75) #*1.25

ax={}

leg_hdl = []

i = 0

#plt.fill_between(x=range(1850,2014), y1=count_unviable_isf_2300.sel(quantile=0.33,scenario='ssp585',time=range(1850,2014)), y2=count_unviable_isf_2300.sel(quantile=0.66,scenario='ssp585',time=range(1850,2014)), color='grey',alpha=0.2)
#plt.fill_between(x=range(2014,2299), y1=count_unviable_isf_2300.sel(quantile=0.33,scenario='ssp585',time=range(2014,2299)), y2=count_unviable_isf_2300.sel(quantile=0.66,scenario='ssp585',time=range(2014,2299)), color='deeppink',alpha=0.2)
#plt.fill_between(x=range(2014,2299), y1=count_unviable_isf_2300.sel(quantile=0.33,scenario='ssp126',time=range(2014,2299)), y2=count_unviable_isf_2300.sel(quantile=0.66,scenario='ssp126',time=range(2014,2299)), color='violet',alpha=0.2)

#qq_list = [1,0.9,0.66,0.33,0.1,0]
qq_list = [0,0.1,0.33,0.66,0.9]
col_tot = 4

for n in range(len(qq_list)-1,0,-1):
    if qq_list[n] == 0.1:
        plt.fill_between(x=range(1850,2015),y1=0,
                                    y2=count_unviable_isf_2300.sel(quantile=qq_list[n],scenario=scen,time=range(1850,2015)), color=cmap_new2(1-0.00001))
        plt.fill_between(x=range(2014,2101),y1=0,
                                            y2=count_unviable_isf_2300.sel(quantile=qq_list[n],scenario=scen,time=range(2014,2101)), color=cmap_new2(1-0.0001))
        plt.fill_between(x=range(1850,2015),y1=count_unviable_isf_2300.sel(quantile=qq_list[n],scenario=scen,time=range(1850,2015)),
                                            y2=count_unviable_isf_2300.sel(quantile=qq_list[n+1],scenario=scen,time=range(1850,2015)), color=cmap_new2(1 - n/col_tot))
        plt.fill_between(x=range(2014,2101),y1=count_unviable_isf_2300.sel(quantile=qq_list[n],scenario=scen,time=range(2014,2101)),
                                            y2=count_unviable_isf_2300.sel(quantile=qq_list[n+1],scenario=scen,time=range(2014,2101)), color=cmap_new2(1 - n/col_tot))
    elif qq_list[n] == 0.9:
        plt.fill_between(x=range(1850,2015),y1=count_unviable_isf_2300.sel(quantile=qq_list[n],scenario=scen,time=range(1850,2015)), color=cmap_new2(1-n/col_tot),
                                            y2=64)
        plt.fill_between(x=range(2014,2101),y1=count_unviable_isf_2300.sel(quantile=qq_list[n],scenario=scen,time=range(2014,2101)), color=cmap_new2(1-n/col_tot),
                                            y2=64)
        
    else:
        plt.fill_between(x=range(1850,2015),y1=count_unviable_isf_2300.sel(quantile=qq_list[n],scenario=scen,time=range(1850,2015)),
                                            y2=count_unviable_isf_2300.sel(quantile=qq_list[n+1],scenario=scen,time=range(1850,2015)), color=cmap_new2(1 - n/col_tot))
        plt.fill_between(x=range(2014,2101),y1=count_unviable_isf_2300.sel(quantile=qq_list[n],scenario=scen,time=range(2014,2101)),
                                            y2=count_unviable_isf_2300.sel(quantile=qq_list[n+1],scenario=scen,time=range(2014,2101)), color=cmap_new2(1 - n/col_tot))

#f.legend()
#f.subplots_adjust(bottom=0.05, wspace=0.1)
plt.xlim(1850,2100)
plt.ylim(0,64)
plt.ylabel('Number of non-viable ice shelves')
plt.xlabel('Time')
f.tight_layout()
sns.despine()
plt.savefig(plot_path + 'nb_non_viable_isf_2300_fill_'+scen+'_only2100.pdf')

NameError: name 'count_unviable_isf_2300' is not defined

In [40]:
scen='ssp585'

### THINK ABOUT THE SIGMA_MOD AND SIGMA_OBS AGAIN

regions = ['Weddell','Bellingshausen','Amundsen','Ross','East 1','East 2','Dronning Maud Land']
colors = ['red','orange','gold','mediumturquoise','maroon','magenta','cornflowerblue','lightseagreen',
          'yellowgreen','royalblue','tomato','darkslateblue','purple','darkgoldenrod']

f = plt.figure()
#f.set_size_inches(8.25, 8.25)
f.set_size_inches(8.25*0.75, 8.25/2*0.75) #*1.25

ax={}

leg_hdl = []

i = 0

#plt.fill_between(x=range(1850,2014), y1=count_unviable_isf_2300.sel(quantile=0.33,scenario='ssp585',time=range(1850,2014)), y2=count_unviable_isf_2300.sel(quantile=0.66,scenario='ssp585',time=range(1850,2014)), color='grey',alpha=0.2)
#plt.fill_between(x=range(2014,2299), y1=count_unviable_isf_2300.sel(quantile=0.33,scenario='ssp585',time=range(2014,2299)), y2=count_unviable_isf_2300.sel(quantile=0.66,scenario='ssp585',time=range(2014,2299)), color='deeppink',alpha=0.2)
#plt.fill_between(x=range(2014,2299), y1=count_unviable_isf_2300.sel(quantile=0.33,scenario='ssp126',time=range(2014,2299)), y2=count_unviable_isf_2300.sel(quantile=0.66,scenario='ssp126',time=range(2014,2299)), color='violet',alpha=0.2)

#qq_list = [1,0.9,0.66,0.33,0.1,0]
qq_list = [0,0.1,0.33,0.66,0.9]
col_tot = 4

for n in range(len(qq_list)-1,0,-1):
    if qq_list[n] == 0.1:
        plt.fill_between(x=range(1850,2015),y1=0,
                                    y2=count_unviable_isf.sel(quantile=qq_list[n],scenario=scen,time=range(1850,2015)), color=cmap_new2(1-0.00001))
        plt.fill_between(x=range(2014,2101),y1=0,
                                            y2=count_unviable_isf.sel(quantile=qq_list[n],scenario=scen,time=range(2014,2101)), color=cmap_new2(1-0.0001))
        plt.fill_between(x=range(1850,2015),y1=count_unviable_isf.sel(quantile=qq_list[n],scenario=scen,time=range(1850,2015)),
                                            y2=count_unviable_isf.sel(quantile=qq_list[n+1],scenario=scen,time=range(1850,2015)), color=cmap_new2(1 - n/col_tot))
        plt.fill_between(x=range(2014,2101),y1=count_unviable_isf.sel(quantile=qq_list[n],scenario=scen,time=range(2014,2101)),
                                            y2=count_unviable_isf.sel(quantile=qq_list[n+1],scenario=scen,time=range(2014,2101)), color=cmap_new2(1 - n/col_tot))
    elif qq_list[n] == 0.9:
        plt.fill_between(x=range(1850,2015),y1=count_unviable_isf.sel(quantile=qq_list[n],scenario=scen,time=range(1850,2015)), color=cmap_new2(1 - n/col_tot),
                                            y2=64)
        plt.fill_between(x=range(2014,2101),y1=count_unviable_isf.sel(quantile=qq_list[n],scenario=scen,time=range(2014,2101)), color=cmap_new2(1 - n/col_tot),
                                            y2=64)
        
    else:
        plt.fill_between(x=range(1850,2015),y1=count_unviable_isf.sel(quantile=qq_list[n],scenario=scen,time=range(1850,2015)),
                                            y2=count_unviable_isf.sel(quantile=qq_list[n+1],scenario=scen,time=range(1850,2015)), color=cmap_new2(1 - n/col_tot))
        plt.fill_between(x=range(2014,2101),y1=count_unviable_isf.sel(quantile=qq_list[n],scenario=scen,time=range(2014,2101)),
                                            y2=count_unviable_isf.sel(quantile=qq_list[n+1],scenario=scen,time=range(2014,2101)), color=cmap_new2(1 - n/col_tot))

#f.legend()
#f.subplots_adjust(bottom=0.05, wspace=0.1)
plt.xlim(1850,2101)
plt.ylabel('Number of non-viable ice shelves')
plt.xlabel('Time')
f.tight_layout()
sns.despine()
plt.savefig(plot_path + 'nb_non_viable_isf_2100_fill_'+scen+'_only2100.pdf')

In [ ]:

### THINK ABOUT THE SIGMA_MOD AND SIGMA_OBS AGAIN

regions = ['Weddell','Bellingshausen','Amundsen','Ross','East 1','East 2','Dronning Maud Land']
colors = ['red','orange','gold','mediumturquoise','maroon','magenta','cornflowerblue','lightseagreen',
          'yellowgreen','royalblue','tomato','darkslateblue','purple','darkgoldenrod']

f = plt.figure()
#f.set_size_inches(8.25, 8.25)
f.set_size_inches(8.25*1.5, 8.25/3*1.5)

ax={}

leg_hdl = []

i = 0

#plt.fill_between(x=range(1850,2014), y1=count_unviable_isf_2300.sel(quantile=0.33,scenario='ssp585',time=range(1850,2014)), y2=count_unviable_isf_2300.sel(quantile=0.66,scenario='ssp585',time=range(1850,2014)), color='grey',alpha=0.2)
#plt.fill_between(x=range(2014,2299), y1=count_unviable_isf_2300.sel(quantile=0.33,scenario='ssp585',time=range(2014,2299)), y2=count_unviable_isf_2300.sel(quantile=0.66,scenario='ssp585',time=range(2014,2299)), color='deeppink',alpha=0.2)
#plt.fill_between(x=range(2014,2299), y1=count_unviable_isf_2300.sel(quantile=0.33,scenario='ssp126',time=range(2014,2299)), y2=count_unviable_isf_2300.sel(quantile=0.66,scenario='ssp126',time=range(2014,2299)), color='violet',alpha=0.2)

for qq in [0.1,0.33,0.66,0.9]:
    plt.plot(range(1850,2014),count_unviable_isf_2300.sel(quantile=qq,scenario='ssp585',time=range(1850,2014)), color=cmap(1-qq), linestyle='dashdot',linewidth=2)
    plt.plot(range(2014,2298),count_unviable_isf_2300.sel(quantile=qq,scenario='ssp585',time=range(2014,2298)), color=cmap(1-qq), linestyle='-',linewidth=2)
    plt.plot(range(2014,2298),count_unviable_isf_2300.sel(quantile=qq,scenario='ssp126',time=range(2014,2298)), color=cmap(1-qq), linestyle='--',linewidth=2)

#plt.plot(range(1850,2014),count_unviable_isf_2300.sel(quantile=0.1,scenario='ssp585',time=range(1850,2014)), color='k', linestyle = '--')
#plt.plot(range(2014,2299),count_unviable_isf_2300.sel(quantile=0.1,scenario='ssp585',time=range(2014,2299)), color='deeppink', linestyle = '--')
#plt.plot(range(2014,2299),count_unviable_isf_2300.sel(quantile=0.1,scenario='ssp126',time=range(2014,2299)), color='violet', linestyle = '--')

#plt.plot(range(1850,2014),count_unviable_isf_2300.sel(quantile=0.9,scenario='ssp585',time=range(1850,2014)), color='k', linestyle = '--')
#plt.plot(range(2014,2299),count_unviable_isf_2300.sel(quantile=0.9,scenario='ssp585',time=range(2014,2299)), color='deeppink', linestyle = '--')
#plt.plot(range(2014,2299),count_unviable_isf_2300.sel(quantile=0.9,scenario='ssp126',time=range(2014,2299)), color='violet', linestyle = '--')

#f.legend()
#f.subplots_adjust(bottom=0.05, wspace=0.1)
plt.xlim(1850,2298)
plt.ylabel('Number of non-viable ice shelves')
plt.xlabel('Time')
f.tight_layout()
sns.despine()
plt.savefig(plot_path + 'nb_non_viable_isf_2300_lines_calving0.pdf')

In [23]:
# NON-VIABILITY SMB
SMB_weighted_yy_2300 = file_viability_info['SMB'].sel(time=range(1850,2300)).sel(model=model_2300).weighted(bay_weights_2300 * sens_weights.sel(model=model_2300))
weighted_quantiles_SMB_2300 = SMB_weighted_yy_2300.quantile([0,0.1,0.33,0.66,0.5,0.9,1],dim=['model','param','m'])
SMB_unviable_times_2300 = weighted_quantiles_SMB_2300 < 0

SMB_viability_diff_2300 = SMB_unviable_times_2300.sel(time=range(2298,1849,-1)).cumsum('time').diff('time')
SMB_limit_max_2300 = SMB_viability_diff_2300.time.where(SMB_viability_diff_2300 == 0).max('time') + 1
SMB_limit_max_2300 = SMB_limit_max_2300.where(SMB_limit_max_2300<2299)
SMB_limit_max_2300 = SMB_limit_max_2300.where(~(np.isnan(SMB_limit_max_2300) & SMB_unviable_times_2300.sel(time=1850)), 1850)

SMB_limit_max_full_2300 = SMB_limit_max_2300.where(np.isfinite(SMB_limit_max_2300), 2305)

SMB_count_unviable_isf_2300 = (SMB_unviable_times_2300.time >= SMB_limit_max_full_2300).sum('Nisf')

In [53]:
scen='ssp585'

### THINK ABOUT THE SIGMA_MOD AND SIGMA_OBS AGAIN

regions = ['Weddell','Bellingshausen','Amundsen','Ross','East 1','East 2','Dronning Maud Land']
colors = ['red','orange','gold','mediumturquoise','maroon','magenta','cornflowerblue','lightseagreen',
          'yellowgreen','royalblue','tomato','darkslateblue','purple','darkgoldenrod']

f = plt.figure()
#f.set_size_inches(8.25, 8.25)
f.set_size_inches(8.25*0.75, 8.25/2*0.75) #*1.25

ax={}

leg_hdl = []

i = 0

#plt.fill_between(x=range(1850,2014), y1=count_unviable_isf_2300.sel(quantile=0.33,scenario='ssp585',time=range(1850,2014)), y2=count_unviable_isf_2300.sel(quantile=0.66,scenario='ssp585',time=range(1850,2014)), color='grey',alpha=0.2)
#plt.fill_between(x=range(2014,2299), y1=count_unviable_isf_2300.sel(quantile=0.33,scenario='ssp585',time=range(2014,2299)), y2=count_unviable_isf_2300.sel(quantile=0.66,scenario='ssp585',time=range(2014,2299)), color='deeppink',alpha=0.2)
#plt.fill_between(x=range(2014,2299), y1=count_unviable_isf_2300.sel(quantile=0.33,scenario='ssp126',time=range(2014,2299)), y2=count_unviable_isf_2300.sel(quantile=0.66,scenario='ssp126',time=range(2014,2299)), color='violet',alpha=0.2)

#qq_list = [1,0.9,0.66,0.33,0.1,0]
qq_list = [0,0.1,0.33,0.66,0.9]
col_tot = 4

for n in range(len(qq_list)-1,0,-1):
    if qq_list[n] == 0.1:
        plt.fill_between(x=range(1850,2015),y1=0,
                                    y2=SMB_count_unviable_isf_2300.sel(quantile=qq_list[n],scenario=scen,time=range(1850,2015)), color=cmap_new2(1-0.00001))
        plt.fill_between(x=range(2014,2298),y1=0,
                                            y2=SMB_count_unviable_isf_2300.sel(quantile=qq_list[n],scenario=scen,time=range(2014,2298)), color=cmap_new2(1-0.0001))
        plt.fill_between(x=range(1850,2015),y1=SMB_count_unviable_isf_2300.sel(quantile=qq_list[n],scenario=scen,time=range(1850,2015)),
                                            y2=SMB_count_unviable_isf_2300.sel(quantile=qq_list[n+1],scenario=scen,time=range(1850,2015)), color=cmap_new2(1 - n/col_tot))
        plt.fill_between(x=range(2014,2298),y1=SMB_count_unviable_isf_2300.sel(quantile=qq_list[n],scenario=scen,time=range(2014,2298)),
                                            y2=SMB_count_unviable_isf_2300.sel(quantile=qq_list[n+1],scenario=scen,time=range(2014,2298)), color=cmap_new2(1 - n/col_tot))
    elif qq_list[n] == 0.9:
        plt.fill_between(x=range(1850,2015),y1=SMB_count_unviable_isf_2300.sel(quantile=qq_list[n],scenario=scen,time=range(1850,2015)), color=cmap_new2(1-n/col_tot),
                                            y2=64)
        plt.fill_between(x=range(2014,2298),y1=SMB_count_unviable_isf_2300.sel(quantile=qq_list[n],scenario=scen,time=range(2014,2298)), color=cmap_new2(1-n/col_tot),
                                            y2=64)
        
    else:
        plt.fill_between(x=range(1850,2015),y1=SMB_count_unviable_isf_2300.sel(quantile=qq_list[n],scenario=scen,time=range(1850,2015)),
                                            y2=SMB_count_unviable_isf_2300.sel(quantile=qq_list[n+1],scenario=scen,time=range(1850,2015)), color=cmap_new2(1 - n/col_tot))
        plt.fill_between(x=range(2014,2298),y1=SMB_count_unviable_isf_2300.sel(quantile=qq_list[n],scenario=scen,time=range(2014,2298)),
                                            y2=SMB_count_unviable_isf_2300.sel(quantile=qq_list[n+1],scenario=scen,time=range(2014,2298)), color=cmap_new2(1 - n/col_tot))

#f.legend()
#f.subplots_adjust(bottom=0.05, wspace=0.1)
plt.xlim(1850,2298)
plt.ylim(0,64)
plt.ylabel('Number of ice shelves reaching SMB < 0')
plt.xlabel('Time')
f.tight_layout()
sns.despine()
plt.savefig(plot_path + 'SMB_nb_non_viable_isf_2300_withoutGISS_fill_'+scen+'.pdf')

Same plots for hydrofracturing

In [22]:
### Colorbar:
cmap = mpl.cm.PuBu #Blues # define the colormap
# extract all colors from the .jet map
cmaplist = [cmap(i) for i in range(cmap.N)]
# force the first color entry to be grey
# cmaplist[0] = (.5, .5, .5, 1.0)

# create the new map
cmap_new = mpl.colors.LinearSegmentedColormap.from_list(
    'Custom cmap', cmaplist, cmap.N)

alpha = 0.8
cmap_with_alpha = cmap(np.arange(cmap_new.N))
cmap_with_alpha[:, -1] = alpha
cmap_new3 = mpl.colors.ListedColormap(cmap_with_alpha)

# define the bins and normalize
bounds = np.arange(6)
norm = mpl.colors.BoundaryNorm(bounds, cmap_new3.N)

In [24]:
scen='ssp585'


regions = ['Weddell','Bellingshausen','Amundsen','Ross','East 1','East 2','Dronning Maud Land']
colors = ['red','orange','gold','mediumturquoise','maroon','magenta','cornflowerblue','lightseagreen',
          'yellowgreen','royalblue','tomato','darkslateblue','purple','darkgoldenrod']

f = plt.figure()
#f.set_size_inches(8.25, 8.25)
f.set_size_inches(8.25*0.75, 8.25/2*0.75) #*1.25

ax={}

leg_hdl = []

i = 0

#plt.fill_between(x=range(1850,2014), y1=count_unviable_isf_2300.sel(quantile=0.33,scenario='ssp585',time=range(1850,2014)), y2=count_unviable_isf_2300.sel(quantile=0.66,scenario='ssp585',time=range(1850,2014)), color='grey',alpha=0.2)
#plt.fill_between(x=range(2014,2299), y1=count_unviable_isf_2300.sel(quantile=0.33,scenario='ssp585',time=range(2014,2299)), y2=count_unviable_isf_2300.sel(quantile=0.66,scenario='ssp585',time=range(2014,2299)), color='deeppink',alpha=0.2)
#plt.fill_between(x=range(2014,2299), y1=count_unviable_isf_2300.sel(quantile=0.33,scenario='ssp126',time=range(2014,2299)), y2=count_unviable_isf_2300.sel(quantile=0.66,scenario='ssp126',time=range(2014,2299)), color='violet',alpha=0.2)

#qq_list = [1,0.9,0.66,0.33,0.1,0]
qq_list = [0.01,0.1,0.33,0.66,0.9]
col_tot = 4

for n in range(len(qq_list)-1,0,-1):
    if qq_list[n] == 0.1:
        plt.fill_between(x=range(1850,2015),y1=0,
                                    y2=count_hflimit_isf_2300.sel(pctl=qq_list[n]*100,scenario=scen,time=range(1850,2015)), color=cmap_new3(1-0.00001))
        plt.fill_between(x=range(2014,2298),y1=0,
                                            y2=count_hflimit_isf_2300.sel(pctl=qq_list[n]*100,scenario=scen,time=range(2014,2298)), color=cmap_new3(1-0.0001))
        plt.fill_between(x=range(1850,2015),y1=count_hflimit_isf_2300.sel(pctl=qq_list[n]*100,scenario=scen,time=range(1850,2015)),
                                            y2=count_hflimit_isf_2300.sel(pctl=qq_list[n+1]*100,scenario=scen,time=range(1850,2015)), color=cmap_new3(1 - n/col_tot))
        plt.fill_between(x=range(2014,2298),y1=count_hflimit_isf_2300.sel(pctl=qq_list[n]*100,scenario=scen,time=range(2014,2298)),
                                            y2=count_hflimit_isf_2300.sel(pctl=qq_list[n+1]*100,scenario=scen,time=range(2014,2298)), color=cmap_new3(1 - n/col_tot))
    elif qq_list[n] == 0.9:
        plt.fill_between(x=range(1850,2015),y1=count_hflimit_isf_2300.sel(pctl=qq_list[n]*100,scenario=scen,time=range(1850,2015)), color=cmap_new3(1-n/col_tot),
                                            y2=64)
        plt.fill_between(x=range(2014,2298),y1=count_hflimit_isf_2300.sel(pctl=qq_list[n]*100,scenario=scen,time=range(2014,2298)), color=cmap_new3(1-n/col_tot),
                                            y2=64)
        
    else:
        plt.fill_between(x=range(1850,2015),y1=count_hflimit_isf_2300.sel(pctl=qq_list[n]*100,scenario=scen,time=range(1850,2015)),
                                            y2=count_hflimit_isf_2300.sel(pctl=qq_list[n+1]*100,scenario=scen,time=range(1850,2015)), color=cmap_new3(1 - n/col_tot))
        plt.fill_between(x=range(2014,2298),y1=count_hflimit_isf_2300.sel(pctl=qq_list[n]*100,scenario=scen,time=range(2014,2298)),
                                            y2=count_hflimit_isf_2300.sel(pctl=qq_list[n+1]*100,scenario=scen,time=range(2014,2298)), color=cmap_new3(1 - n/col_tot))

#f.legend()
#f.subplots_adjust(bottom=0.05, wspace=0.1)
plt.xlim(1850,2298)
plt.ylim(0,64)
plt.ylabel('Number of ice shelves at risk of hydrofracturing')
plt.xlabel('Time')
f.tight_layout()
sns.despine()
plt.savefig(plot_path + 'nb_hydrofrac_isf_2300_withoutGISS_fill_'+scen+'.pdf')

In [59]:
hf_limit.sel(pctl=10,scenario='ssp585').load()

<xarray.DataArray 'hydrofrac_limit' (Nisf: 64)>
array([2500, 2500, 2500, 2500, 2500, 2500, 2500, 2500, 2500, 2500, 2500,
       2500, 2500, 2500, 2500, 2500, 2500, 2500, 2500, 2500, 2500, 2500,
       2500, 2500, 2500, 2500, 2500, 2500, 2500, 2500, 2500, 2500, 2500,
       2500, 2500, 2500, 2500, 2500, 2500, 2500, 2500, 2500, 2500, 2500,
       2500, 2500, 2500, 2500, 2500, 2500, 2500, 2500, 2500, 2500, 2500,
       2500, 2500, 2500, 2500, 2500, 2500, 2500, 2500, 2500])
Coordinates:
  * Nisf      (Nisf) int64 10 11 12 13 14 15 16 17 ... 68 69 70 71 72 73 74 75
    pctl      int64 10
    scenario  <U6 'ssp585'

In [57]:
hf_limit.sel(pctl=33,scenario='ssp585') < 2300

<xarray.DataArray 'hydrofrac_limit' (Nisf: 64)>
array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])
Coordinates:
  * Nisf      (Nisf) int64 10 11 12 13 14 15 16 17 ... 68 69 70 71 72 73 74 75
    pctl      int64 33
    scenario  <U6 'ssp585'

In [53]:
hf_limit.sel(pctl=66,scenario='ssp585',Nisf=11).load()

<xarray.DataArray 'hydrofrac_limit' ()>
array(2117)
Coordinates:
    Nisf      int64 11
    pctl      int64 66
    scenario  <U6 'ssp585'

In [54]:
hf_limit.sel(pctl=90,scenario='ssp585',Nisf=11).load()

<xarray.DataArray 'hydrofrac_limit' ()>
array(2103)
Coordinates:
    Nisf      int64 11
    pctl      int64 90
    scenario  <U6 'ssp585'

In [64]:
Nisf_very_likely = limit_max_full_2300.Nisf.where(limit_max_full_2300.sel(quantile=0.1,scenario='ssp585') > SMB_limit_max_full_2300.sel(quantile=0.1,scenario='ssp585'), drop=True)
Nisf_likely = limit_max_full_2300.Nisf.where(limit_max_full_2300.sel(quantile=0.33,scenario='ssp585') > SMB_limit_max_full_2300.sel(quantile=0.33,scenario='ssp585'), drop=True)

In [69]:
for kisf in sorted_isf_rignot:
    if kisf not in Nisf_likely:
        print(file_isf['isf_name'].sel(Nisf=kisf).values)

Stancomb Brunt
Atka
Jelbart
Fimbul
Vigrid
Lazarev
Borchgrevink
Prince Harald
Shackleton
Conger/Glenzer
Holmes
Cook
Mariner
Ross
Swinburne
Sulzberger
Nickerson
Getz
Abbot
Venable
Ferrigno
Stange
Wilkins
George VI
Wordie


In [73]:
(file_isf['ISF_mask'] == Nisf_likely).sum('Nisf').plot()